# 라이브러리

In [1]:
import pandas as pd

# 데이터

In [2]:
df = pd.read_csv('BIT_2023_1차.csv')
df.head(3)

,time,Open,High,Low,Close,Volume,returns,volatility,price,quote_qty,is_buyer_maker,quote_qty_cal
0,2023-01-01 0:00,16537.5,16540.9,16504.0,16527.0,5381.399,NaN,NaN,16524.52574,7264.129209,16494,88925066.30
1,2023-01-01 1:00,16527.1,16554.3,16524.1,16550.4,3210.826,0.001416,NaN,16537.21599,6819.889969,8705,53098123.05
2,2023-01-01 2:00,16550.5,16557.1,16534.8,16542.4,2399.668,-0.000483,NaN,16545.81814,6030.420093,8468,39704470.32


# Stochastic Oscillator

'%K' 값은 최근 N일간의 종가(C)가 최고가(H)와 최저가(L) 사이에서 어느 정도 위치하고 있는지를 나타내는 지표

Stochastic Oscillator의 %D는 %K의 이동평균값

기간(period)은 사용자의 요구에 따라 조정

m은 Stochastic Oscillator의 %D 값을 계산할 때 사용되는 이동평균 기간   
= m 매개변수는 %D 값을 계산할 때 사용할 이동평균의 기간을 지정하는 역할
- 이동평균은 시계열 데이터에서 변동을 완화하고 추세를 확인하는 데 도움이 되는 통계적 기법
- 예를 들어, m=3로 설정되어 있다면, %K 값의 최근 3개 값의 평균을 계산하여 %D 값을 얻는다

Stochastic Oscillator는 %K와 %D 두 가지 지표로 구성
- %K는 주가의 현재 위치를 나타내는 지표
- %D는 %K 값의 이동평균을 나타내어 주가의 추세를 확인하는 데 활용

> 따라서, 주어진 코드에서 m 값을 조정함으로써 %D 값의 이동평균 기간을 변경할 수 있다. 일반적으로 3일 이동평균이 많이 사용되지만, 분석 목적이나 시장의 특성에 따라 다른 값으로 설정할 수도 있다.

In [3]:
# Stochastic Oscillator 함수
def cal_stoc_os(df, period=14, m=3): # period = 14 or 20
    
    # 최근 N일간의 최고가(H)와 최저가(L)
    df['highest_high'] = df['High'].rolling(window = period).max()
    df['lowest_low'] = df['Low'].rolling(window = period).min()
    
    # %K 계산
    df['%K'] = (df['Close'] - df['lowest_low']) / (df['highest_high'] - df['lowest_low']) * 100
    
    # %D 계산
    df['%D'] = df['%K'].rolling(m).mean()

    return df

In [4]:
# Stochastic Oscillator 계산 실행
df = cal_stoc_os(df)

In [5]:
# 결과 출력
print(df[['time', '%K', '%D']])

                  time         %K         %D
0      2023-01-01 0:00        NaN        NaN
1      2023-01-01 1:00        NaN        NaN
2      2023-01-01 2:00        NaN        NaN
3      2023-01-01 3:00        NaN        NaN
4      2023-01-01 4:00        NaN        NaN
...                ...        ...        ...
8755  2023-12-31 19:00  59.875948  62.067090
8756  2023-12-31 20:00  47.162162  57.553939
8757  2023-12-31 21:00  31.210526  46.082879
8758  2023-12-31 22:00  21.797164  33.389951
8759  2023-12-31 23:00  31.333967  28.113886

[8760 rows x 3 columns]


## 결측치

Stochastic Oscillator의 '%K' 값이 결측치인 NaN으로 표기되는 이유?
- 주어진 기간(period) 동안 최고가(H)와 최저가(L)가 동일한 값이거나, 최고가와 최저가를 계산할 수 없는 경우   
- 이런 경우에는 '%K' 값을 정확하게 계산할 수 없으므로 결측치로 처리

%D가 NaN으로 표기되는 이유?
- 주어진 데이터의 시작 부분에서 rolling window의 크기에 맞는 이전 데이터가 없기 때문
- Stochastic Oscillator의 %D는 %K의 이동평균값으로 계산
- 이동평균을 계산하기 위해서는 최소한 이동평균의 기간만큼의 이전 데이터가 필요

In [6]:
df['%K'].isna().sum()

128

In [10]:
df['%D'].isna().sum()

144

1) 주가 데이터에서 최고가와 최저가가 동일한 경우는 주로 가격이 변동이 없는 경우이거나, 데이터의 불완전함 등의 이유로 발생할 수 있다. 
- 이러한 상황에서는 '%K' 값을 계산할 수 없으므로 NaN으로 표기된다

2) 이를 해결하기 위해서는 주어진 데이터를 검토하고, 필요한 경우 최고가와 최저가를 수정하거나 보완해야 할 것이다. 
- 예를 들어, 최고가와 최저가가 동일한 경우에는 일정한 값을 사용하거나, 다른 방법으로 값을 대체하여 결측치를 처리할 수 있다.

3) 결측치 처리에는 다양한 방법이 있으며, 데이터의 특성과 분석 목적에 따라 적합한 방법을 선택해야 한다. 
- 따라서, 주어진 데이터에서 '%K' 값이 NaN으로 표기되는 정확한 이유를 파악하고, 데이터를 적절하게 처리하여 원하는 결과를 얻을 수 있도록 조치

In [9]:
df.tail(3)

,time,Open,High,Low,Close,Volume,returns,volatility,price,quote_qty,is_buyer_maker,quote_qty_cal,highest_high,lowest_low,%K,%D
8757,2023-12-31 21:00,42599.2,42717.0,42558.2,42558.9,3794.010,-0.000944,0.002673,42634.52352,13367.47978,21732,161755808.6,42951.0,42381.0,31.210526,46.082879
8758,2023-12-31 22:00,42559.0,42629.5,42111.9,42294.8,11952.346,-0.006206,0.003008,42405.77979,19111.64804,67159,506848552.4,42951.0,42111.9,21.797164,33.389951
8759,2023-12-31 23:00,42294.8,42380.1,42083.1,42314.0,11676.465,0.000454,0.002975,42244.20465,16705.04539,58180,493262977.0,42820.0,42083.1,31.333967,28.113886


quote_qty_cal, highest_high, lowest_low	, %K, %D 값이 잘 반영된 것을 확인할 수 있다.